# Homeworks

For the course of Statistical Foundations of Machine Learning, two homeworks had to be done with a notebook description that will be useful for the presentation during the exam.
The selected homeworks are homework number 1 (over the Perceptron Learning Algorithm) and homework number 4 (over overfitting and Regularization with Weight Decay).

The goals of those homeworks are to implement both techniques and to answer questions over them.

# Homework 1 : The Perceptron Learning Algorithm

Perceptron is a supervised learning algorithm of linear classifiers. This kind of algorithm uses a training set, which is made of output depending on inputs examples, and try to produce an inferred function by analysing the training set. In this work, the perceptron only has one layer and one output. It means that all the inputs are directly connected to the output which is 1 or -1. This algorithm also uses the notion of weight.

## The goal

After creating a random target function _f_ and a data set _D_. We have to implement the algorithm to see how it works. Inputs are tuple of two values in _X_ = [-1,1] x [-1,1]. Since we implement a classifier, the output variable takes class labels and not a continious value which is the case for linear regression.

 
### The Training Set

So the first step is to create the Training Set ([_**x**(n),y(n)_)] which is made of point.
A target function is generated based on two randoms points in _X_ (the attributes of the data, d=2). For each point in the training set, we have 3 values : the input made of two values and the output value. Then _**x**(n)_ points are generated and according to the point values, the corresponding _y_ is calculated. To do so, we only have to know if the point is in the left (1) or the right (-1) part of the target function.

In [ ]:
def constructTargetFunction(self, N):
    # Construct random target function f
    rPA = self.getRandomPoint()
    rPB = self.getRandomPoint()
    self.target = [rPA, rPB]
    
def constructTrainingSet(self, N):
    self.dataset = []
    
    # Construct training set : [ [point(x,y), [+, -]], ...]
    for i in range(N):
        x = [self.getRandom() for i in range(self.d)]
        self.dataset.append([x, self.getY(x)])

def getY(self, point):
        if ( ((self.target[1][0]-self.target[0][0])*(point[1]-self.target[0][1])) - ((self.target[1][1]-self.target[0][1])*(point[0]-self.target[0][0])) > 0): 
            return 1
        return -1


### Perception Learning Algorithm

As long as the training set is built up, the training function has to be trained. Indeed the training function which mainly depends on the weight vector tries to converge to the target function by adjusting this weight vector. Each attribute of our data has a specific weight. This training function is called **_h_(x)**.

**_h_**(x) = sign((w^(t))\*x).

At first, the weight vector is initialised to zero. So, the training function and the target function are badly different. This technique consists of picking at each step a _misclassified_ point and then updates the weight vector depending on the previous picked point. The way to update the weight vector is :

w <- w + y(n)**x**(n) where **x**(n) is the _misclassified_ point.

A point is considered as _misclassified_ as long as its value through the training function (h(x(i))) is different from the real one (y(i)).

The following code executes a typical run of the **Perceptron Learning Algorithm (_PLA_)**

In [ ]:
def PLA(self, maxIteration):
    """" Executes a run of PLA """
    self.initWeightVector()
    iterationNbr = 0
    while iterationNbr < maxIteration : # The classification is bounded to 5000 turns
        # Construct the misclassified point vector
        self.misclassifiedVector = self.constructMisclassifiedVector()

        # As long as misclassified point exists
        if (len(self.misclassifiedVector) == 0):
            break
        mcX = self.pickMisclassified()

        #Update the weight vector
        self.updateW(mcX)
        iterationNbr+=1

    return iterationNbr

To have a great idea of the efficiency of **PLA**. We will run the algorithm over 1000 turns and by means of means, we will get average values for e.g. number of iteration or misclassified point probability:

In [6]:
def run(self, N, turn):
    """ Run (turn) turns of PLA with N points"""
    probTot = 0
    convergeTot = 0

    for i in range(turn):
        self.constructTargetFunction()
        self.constructTrainingSet(N)

        convergeTot+=self.PLA(2000)
        probTot+=self.computeMissedProbability(turn)

    avgConverge = convergeTot/turn
    missedProb = probTot/turn
    print("Average iteration convergence: "+str((avgConverge)))
    print("Missed probability: "+str((missedProb)))

    self.normalizeConfusionMatrix(turn*turn)

    return avgConverge, missedProb, self.confusionMatrix
    
def computeMissedProbability(self, turn):
    """ Computes the probability that a point is misclassified """
    probTmp = 0
    for j in range(turn):
        p = self.getRandomPoint()
        y = self.getY(p)
        estimatedY = self.getH(p)

        if(y != estimatedY):
            probTmp+=1

        self.setConfusionClass(y,estimatedY)

    return probTmp/turn

def setConfusionClass(self, y, h):
    """ Adds a confusion in the matrix """
    if (y==h): # Good estimation => True
        if(y==1): # Should have been 1 => true positive
            self.confusionMatrix['tp']+=1
        else:
            self.confusionMatrix['tn']+=1
    else: # Bad estimation => False
        if(y==1): # Should have been 1 => false negative
            self.confusionMatrix['fn']+=1
        else:
            self.confusionMatrix['fp']+=1

def normalizeConfusionMatrix(self, turn):
    for key in self.confusionMatrix:
        self.confusionMatrix[key]/=turn

## Questions

In [5]:
from perceptron import Perceptron

perceptron = Perceptron(d=2)
#print("N = 10")
perceptron.run(10, 1000)

Average iteration convergence: 9.811
Missed probability: 0.10759400000000001


**7.** For N = 10, how many iterations does it take on average for the PLA to converge ?

>**[b] 15**

**8.** Which probability is the closest to missed probability ?

>**[c] 0.1**

In [4]:
#print("N = 100")
perceptron.run(100, 1000)

Average iteration convergence: 101.915
Missed probability: 0.013002999999999959


**9.** For N = 10, how many iterations does it take on average for the PLA to converge ?

>**[b] 100**

**10.** Which probability is the closest to missed probability ?

>**[b] 0.01**


## Analysis

### Evolution 

The two factors that we just studied are the number of iteration and the misclassification of point probability over the length of the training set **N**. The two questions was about using _N=10_ and _N=100_. So it could interresting to see how the length affects those factors from **N** in a range from 0 to 100.


<div>
    <img width="49%" style="border: 1px solid black;margin:0" src="images/nEvolutionMisclassifiedprobabilityOk.png" align="left">
    <img width="49%" style="border: 1px solid black;margin:0" src="images/nEvolutionNumberofiterationOk.png" align="right">
</div>

We can easily see that increasing the training set length tend to slow down PLA's time execution. In opposite, increasing the training set length make the classifier more efficient. Indeed we can easily see on the first figure that the misclassified probability tend to 0 for **N** = 0. As long as the two factors go in different way. It could be interesting to evaluate the most effecient length **N** for the training set. Since the number of iteration is linear, a quick analysis is to compares slopes at both points of **N** and pick the first one when they are going to converge (especially the missed classification probability because of the linearity of the other one). Then, we can see on the following figure that an efficient **N** could be _20_

<div>
    <img width="50%" style="border: 1px solid black;padding:0 10px 0 0" src="images/nEvolution_slope.png">
</div>

### Confusion Matrix

A **confusion matrix** is a tool to evaluate the quality of a classifier.<br />
Each row represent the number of occurences predicted by the classifier and each column represent the number of real occurences. It usually use the number of occurences but since we run several turns, we will directly use the probability over a data set of 100 points.<br />

Based on this matrix, we can use the kappa score to quantify the quality of the classified.
It exists several interpretation of the Cohen’s kappa. We will use the one illustrated in _[Marry L. McHugh. Interrater reliability: the kappa statistic. Biochemia Medica 2012;22(3):276-82]_

#### N = 10

Confusion Matrix = ['fp': 0.054089, 'fn': 0.059407, 'tn': 0.440985, 'tp': 0.445519]

| Real/Predicted   | 1  | -1  |
|---|---|---|
| 1  | 45%  | 6%  | 
| -1 | 5%  | 44%  |

+ **Ground truth:** 50, 50<br />
+ **Machine learning:** 51, 49<br />
+ **Total:** (100)<br />
<br />
+ **Observed Accuracy:** ((45+44) / 100) = 0.89<br />
+ **Expected Accuracy:** ((50\*51/100) + (50\*49/100)) / 100 = 0.50<br />
+ **Kappa:** (0.89 - 0.50) / (1 - 0.50) = 0.78<br />
<br />
+ **Result:** between **Moderate** (.60-.79) and **Strong** (.80-.90)<br />

#### N = 100

Confusion Matrix = ['tn': 0.492615, 'fp': 0.007225, 'tp': 0.493233, 'fn': 0.006927]

| Real/Predicted   | 1  | -1  |
|---|---|---|
| 1  | 49%  | 1%  | 
| -1 | 1%  | 49%  |

+ **Ground truth:** 50, 50<br />
+ **Machine learning:** 50, 40<br />
+ **Total:** (100)<br />
<br />
+ **Observed Accuracy:** ((49+49) / 100) = 0.98<br />
+ **Expected Accuracy:** ((50\*50/100) + (50\*40/100)) / 100 = 0.45<br />
+ **Kappa:** (0.98 - 0.45) / (1 - 0.45) = 0.96<br />
<br />
+ **Result:** **Almost Perfect** (above .96)<br />

## Homework 6 : Overfitting and Regularization with Weight Decay.

One of the most common tasks in machine learning is trying to find the target function of a training data set. Based on this target function, we can make reliable predictions on untrained data. The main goal is thus to **fit** the target function. Overfitting occurs when the model we are using to fit the target function is too much complex.


There are **two cures** for overfitting :

    1. Regularization
    2. Validation

In this work, we only focus on **Regularization**.
